In [30]:
from Random_search_cv import Random_search_cv,create_example_param_distributions
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from tree import DecisionTree
from random_forest import random_forest
from Qrng import qml_random_choice, NumberGenerator
import time

In [31]:
data = load_breast_cancer()
X,y=data.data,data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


In [32]:
def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred)/len(y_true)

In [33]:
d_t=DecisionTree()
r_f=random_forest()

param_distributions=create_example_param_distributions()
random= NumberGenerator(20**2)
R_num=random.generate_unbiased()
o_r_f= Random_search_cv(random_forest, param_distributions, cv=5, n_iter=5, scoring='accuracy',
                                      random_state=R_num)

In [34]:
start_time = time.time()
d_t.fit(X_train, y_train)
y_pred = d_t.predict(X_test)

acc = accuracy(y_test, y_pred)
print(f"accuracy: {acc*100:.4f}%")
print(y_pred-y_test)
end_time = time.time()
print(f"Time: {end_time-start_time}s")

accuracy: 94.2982%
[ 0  0  0  0  0  0  0  0 -1  0  0  0  0  1  0  0  0  0  0  0  1  0  0  0
  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  1  0  0  0  0  1  0  1  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0]
Time: 0.7010989189147949s


In [35]:
start_time = time.time()
r_f.fit(X_train, y_train)
y_pred = r_f.predict(X_test)

acc = accuracy(y_test, y_pred)
print(f"accuracy: {acc*100:.4f}%")
print(y_pred-y_test)
end_time = time.time()
print(f"Time: {end_time-start_time}s")

accuracy: 96.4912%
[ 0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  1  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0]
Time: 43.400830030441284s


In [36]:
start_time = time.time()
o_r_f.fit(X, y)
best_rf = o_r_f.get_best_estimator()

selected_features = o_r_f.get_quantum_feature_selection(X, n_features=10)
X_selected = X[:, selected_features]

Iteration 1/5: Score = 0.9315 (+/- 0.0279)
  Parameters: {'n_trees': 16, 'max_depth': 136, 'min_sample_split': 151, 'n_features': 45}
Iteration 2/5: Score = 0.9139 (+/- 0.0257)
  Parameters: {'n_trees': 131, 'max_depth': 1, 'min_sample_split': 146, 'n_features': 60}
Iteration 3/5: Score = 0.9332 (+/- 0.0154)
  Parameters: {'n_trees': 81, 'max_depth': 11, 'min_sample_split': 171, 'n_features': 140}
Iteration 4/5: Score = 0.9525 (+/- 0.0071)
  Parameters: {'n_trees': 101, 'max_depth': 106, 'min_sample_split': 11, 'n_features': 35}
Iteration 5/5: Score = 0.9086 (+/- 0.0302)
  Parameters: {'n_trees': 1, 'max_depth': 101, 'min_sample_split': 141, 'n_features': 70}


In [37]:
print(f"Best parameters: {o_r_f.best_params_}")
print(f"Best accuracy: {o_r_f.best_score_:.4f}")


Best parameters: {'n_trees': 101, 'max_depth': 106, 'min_sample_split': 11, 'n_features': 35}
Best accuracy: 0.9525


In [39]:
final_rf = random_forest(**o_r_f.best_params_)
final_rf.fit(X, y)
y_pred = r_f.predict(X_test)

acc = accuracy(y_test, y_pred)
print(f"accuracy: {acc*100:.4f}%")
print(y_pred-y_test)
end_time = time.time()
print(f"Time: {end_time-start_time}s")

accuracy: 96.4912%
[ 0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  1  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0]
Time: 794.4552283287048s
